## Wikipedia Article Retrieval Tool using Open AI

This section defines a simple Python function, `get_article`, that uses the `wikipedia` Python package to search for and retrieve the content of a Wikipedia article based on a search term. This function will be used as a tool for the agent to fetch up-to-date information from Wikipedia.

In [ ]:
import wikipedia
from typing import Optional
import json

def get_article(search_term: str) -> str:
    """
    Retrieve Wikipedia article content.

    Args:
        search_term: Search query for Wikipedia

    Returns:
        Article content or error message
    """
    try:
        if not search_term or not search_term.strip():
            return "Error: Search term cannot be empty."

        results = wikipedia.search(search_term)

        if not results:
            return f"No Wikipedia articles found for '{search_term}'."

        first_result = results[0]
        page = wikipedia.page(first_result, auto_suggest=False)
        return page.content

    except wikipedia.exceptions.DisambiguationError as e:
        return f"The search term '{search_term}' is ambiguous. Please be more specific. Options: {', '.join(e.options[:5])}"

    except wikipedia.exceptions.PageError:
        return f"No Wikipedia page found for '{search_term}'."

    except Exception as e:
        return f"Error retrieving article: {str(e)}"

## Example: Fetching and Previewing Wikipedia Articles

Here, we demonstrate how to use the `get_article` function to retrieve and preview the content of Wikipedia articles for various search terms, such as "Avengers: Doomsday", "Nezha 2", "History of Malaysia", and "Iron Man". Only a preview of the article content is printed for brevity.

In [ ]:
article = get_article("Avengers: Doomsday")
# print(article[:1000]) 
# article is very long, so let's just print a preview

from IPython.display import display, HTML

article = get_article("Avengers: Doomsday")
html_content = f"""
<div style="background-color: #f8f9fa; padding: 20px; border-radius: 8px; border-left: 4px solid #007bff;">
    <h3 style="color: #007bff; margin-top: 0;">Avengers: Doomsday Answer Preview</h3>
    <p style="line-height: 1.6; text-align: justify;">{article[:1000]}...</p>
    <small style="color: #6c757d;">(Showing first 1000 characters)</small>
</div>
"""
display(HTML(html_content))

In [ ]:
article = get_article("Nezha 2")
print(article[:500]) # article is very long, so let's just print a preview

In [ ]:
article = get_article("History of Malaysia")
print(article[:3000]) #article is super long so let's just print a preview

In [ ]:
article = get_article("Iron Man")
print(article[:1000]) #article is super long so let's just print a preview

## Tool Schema Definition for OpenAI Function Calling

This cell defines a tool schema dictionary, `article_search_tool`, which describes the Wikipedia retrieval tool in a format compatible with OpenAI's function calling API. The schema includes:

- **Tool type**: Specifies this as a "function" for OpenAI's API
- **Function name**: Identifies the tool as "get_article"
- **Description**: Explains the tool's purpose for retrieving Wikipedia articles
- **Parameters**: Defines the input structure with required search terms
- **Required fields**: Ensures proper parameter validation

This schema enables GPT models to understand when and how to use the Wikipedia article retrieval functionality.

In [ ]:
article_search_tool = {
    "type": "function",  # This is required for OpenAI
    "function": {
        "name": "get_article",
        "description": "A tool to retrieve an up to date Wikipedia article.",
        "parameters": {
            "type": "object",
            "properties": {
                "search_term": {
                    "type": "string",
                    "description": "The search term to find a wikipedia article by title"
                }
            },
            "required": ["search_term"]
        }
    }
}

In [ ]:
def extract_tool_calls(response):
    """Extract tool calls from OpenAI response."""
    if response.choices[0].message.tool_calls:
        return response.choices[0].message.tool_calls
    return None

## Setting Up OpenAI Client and Making a Tool-Use Request

This section initializes the OpenAI API client and demonstrates a complete tool-use workflow with GPT-4o. The implementation includes:

- **Client Initialization**: Setting up the OpenAI client with proper authentication
- **Tool Schema Definition**: Defining the Wikipedia article retrieval tool for OpenAI's function calling API
- **Two-Stage API Calls**: First call to request tool use, second call to provide final answer
- **Tool Execution**: Implementing the actual Wikipedia article retrieval functionality
- **Beautiful Display**: Using IPython HTML to present results with professional styling

The workflow handles questions that require external information (e.g., "What is the box office for Nezha 2?") by automatically detecting when tool use is needed, executing the Wikipedia search, and presenting the results in an elegant, formatted display.

In [ ]:
from typing import Optional
from IPython.display import display, HTML

def display_tool_result(question: str, answer: str, tool_used: Optional[str] = None, search_term: Optional[str] = None) -> None:
    """Display AI response with beautiful HTML formatting."""
    html_content = f"""
    <div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; max-width: 800px; 
                margin: 20px auto; background: white; border-radius: 15px; 
                box-shadow: 0 10px 30px rgba(0,0,0,0.15); overflow: hidden;">
        <!-- Header -->
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                    color: white; padding: 25px; text-align: center;">
            <h1 style="margin: 0; font-size: 24px;">🤖 GPT-4o-mini Response</h1>
            <p style="margin: 10px 0 0 0; opacity: 0.9;">Powered by OpenAI with Tool Assistance</p>
        </div>
        
        <!-- Question -->
        <div style="padding: 20px 25px 10px;">
            <h3 style="color: #2c3e50; margin: 0;">❓ Question</h3>
            <div style="background: #f8f9fa; padding: 15px; border-radius: 8px; 
                        margin-top: 10px; border-left: 4px solid #3498db;">
                <p style="margin: 0; font-size: 16px; color: #34495e;">{question}</p>
            </div>
        </div>
    """
    
    if tool_used and search_term:
        html_content += f"""
        <div style="padding: 10px 25px;">
            <h3 style="color: #2c3e50; margin: 0;">🔧 Tool Used</h3>
            <div style="background: #fff3cd; padding: 15px; border-radius: 8px; 
                        margin-top: 10px; border-left: 4px solid #f39c12;">
                <p style="margin: 0; font-weight: 600; color: #856404;">
                    <strong>Tool:</strong> {tool_used}<br>
                    <strong>Search Term:</strong> {search_term}
                </p>
            </div>
        </div>
        """
    
    html_content += f"""
        <div style="padding: 10px 25px 25px;">
            <h3 style="color: #2c3e50; margin: 0;">💬 Answer</h3>
            <div style="background: #d4edda; padding: 20px; border-radius: 8px; 
                        margin-top: 10px; border-left: 4px solid #28a745;">
                <p style="margin: 0; font-size: 16px; line-height: 1.8; color: #155724; 
                           text-align: justify;">{answer}</p>
            </div>
        </div>
        
        <!-- Footer -->
        <div style="background: #2c3e50; color: white; padding: 15px 25px; 
                    text-align: center; font-size: 12px;">
            <p style="margin: 0;">Generated with OpenAI GPT-4o-mini and IPython HTML Display</p>
        </div>
    </div>
    """
    
    display(HTML(html_content))

## Model Follow-up: Final Answer Generation

This section sends the updated conversation (including the tool result) back to the model, prompting it to generate a final answer that incorporates the information retrieved from Wikipedia.

In [ ]:
# Initialize OpenAI client
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

In [ ]:
# Initialize messages with a clean state
messages = [{"role": "user", "content": "What is the box office for Nezha 2"}]

# Now run the complete workflow with proper message structure
try:
    # First API call
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=1000,
        tools=[article_search_tool]
    )
    
    # Handle tool calls with proper message structure
    if response.choices[0].message.tool_calls:
        # Add assistant's message with tool_calls FIRST
        messages.append({
            "role": "assistant",
            "content": response.choices[0].message.content,
            "tool_calls": response.choices[0].message.tool_calls
        })
        
        # Execute tool
        tool_call = response.choices[0].message.tool_calls[0]
        tool_name = tool_call.function.name
        tool_input = json.loads(tool_call.function.arguments)
        
        if tool_name == "get_article":
            search_term = tool_input["search_term"]
            wiki_result = get_article(search_term)
            
            # Add tool response AFTER the assistant message
            tool_response = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": wiki_result
            }
            messages.append(tool_response)
            
            # Final API call (without tools parameter)
            follow_up_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                max_tokens=1000
            )
            
            # Get final answer
            final_answer = follow_up_response.choices[0].message.content
            print("Final Answer:", final_answer)
    
except Exception as e:
    print(f"Error: {str(e)}")
    print("Messages at error:", messages)

## Agentic Tool Use: Full Question-Answering Loop

This cell defines a reusable function, `answer_question`, that demonstrates the full agentic tool-use loop: sending a question to the model, detecting tool use, executing the tool, sending the result, and printing the model's final answer.

In [ ]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import wikipedia

# Make sure client is initialized
load_dotenv()
if 'client' not in globals():
    client = OpenAI()

# Define the Wikipedia article retrieval function
# Define the tool schema for OpenAI

def answer_question(question):
    """
    Answer a question using OpenAI GPT-4o with tool assistance
    """
    messages = [{"role": "user", "content": question}]
    
    try:
        # First API call
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_completion_tokens=1000,
        
        # Handle tool calls
        if response.choices[0].message.tool_calls:
            # Add assistant's message with tool_calls (must include the entire message object)
            assistant_message = response.choices[0].message
            messages.append({
                "role": "assistant",
                "content": assistant_message.content or "",  # Use empty string if content is None
                "tool_calls": [
                    {
                        "id": tc.id,
                        "type": tc.type,
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments
                        }
                    }
                    for tc in assistant_message.tool_calls
                ]
            })
            
            # Execute ALL tool calls (not just the first one!)
            for tool_call in response.choices[0].message.tool_calls:
                tool_name = tool_call.function.name
                tool_input = json.loads(tool_call.function.arguments)
                
                if tool_name == "get_article":
                    search_term = tool_input["search_term"]
                    print(f"🔍 Searching Wikipedia for: {search_term}")
                    wiki_result = get_article(search_term)
                    
                    # Add tool response for THIS specific tool call
                    tool_response = {
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": wiki_result
                    }
                    messages.append(tool_response)
                
            # Final API call (after all tool responses are added)
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                max_completion_tokens=1000
            )
        
        # Get final answer
        final_answer = response.choices[0].message.content
        return final_answer
        
    except Exception as e:
        return f"Error: {str(e)}"

# Now you can call it
result = answer_question("What are the names of all the Avengers films that are confirmed in the Marvel Cinematic Universe?")
print("Answer:", result)

In [ ]:
import json
from IPython.display import display, HTML

def display_answer_with_html(question, answer):
    """
    Display the question and answer with beautiful HTML styling
    """
    html_content = f"""
    <div style="
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        max-width: 800px;
        margin: 20px auto;
        background: white;
        border-radius: 15px;
        box-shadow: 0 10px 30px rgba(0,0,0,0.15);
        overflow: hidden;
    ">
        <!-- Header -->
        <div style="
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 25px;
            text-align: center;
        ">
            <h1 style="margin: 0; font-size: 24px;">🤖 AI Response</h1>
            <p style="margin: 10px 0 0 0; opacity: 0.9;">Powered by GPT-4o with Tool Assistance</p>
        </div>
        
        <!-- Question -->
        <div style="padding: 20px 25px 10px;">
            <h3 style="color: #2c3e50; margin: 0;">❓ Question</h3>
            <div style="
                background: #f8f9fa;
                padding: 15px;
                border-radius: 8px;
                margin-top: 10px;
                border-left: 4px solid #3498db;
            ">
                <p style="margin: 0; font-size: 16px; color: #34495e;">{question}</p>
            </div>
        </div>
        
        <!-- Answer -->
        <div style="padding: 10px 25px 25px;">
            <h3 style="color: #2c3e50; margin: 0;">💬 Answer</h3>
            <div style="
                background: #d4edda;
                padding: 20px;
                border-radius: 8px;
                margin-top: 10px;
                border-left: 4px solid #28a745;
            ">
                <p style="
                    margin: 0; 
                    font-size: 16px; 
                    line-height: 1.8; 
                    color: #155724;
                    text-align: justify;
                ">{answer}</p>
            </div>
        </div>
        
        <!-- Footer -->
        <div style="
            background: #2c3e50;
            color: white;
            padding: 15px 25px;
            text-align: center;
            font-size: 12px;
        ">
            <p style="margin: 0;">Generated with OpenAI GPT-4o and IPython HTML Display</p>
        </div>
    </div>
    """
    
    display(HTML(html_content))

def answer_question(question):
    """
    Answer a question using OpenAI GPT-4o with tool assistance
    """
    # Start with a clean messages array
    messages = [{"role": "user", "content": question}]
    
    try:
        # First API call
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=1000,
            tools=[article_search_tool]
        )
        
        # Handle tool calls
        if response.choices[0].message.tool_calls:
            # Add assistant's message with tool_calls
            messages.append({
                "role": "assistant",
                "content": response.choices[0].message.content,
                "tool_calls": response.choices[0].message.tool_calls
            })
            
            # Execute ALL tool calls (not just the first one)
            for tool_call in response.choices[0].message.tool_calls:
                tool_name = tool_call.function.name
                tool_input = json.loads(tool_call.function.arguments)
                
                if tool_name == "get_article":
                    search_term = tool_input["search_term"]
                    print(f"🔍 Searching Wikipedia for: {search_term}")
                    wiki_result = get_article(search_term)
                    
                    # Add tool response for THIS specific tool call
                    tool_response = {
                        "role": "tool",
                        "tool_call_id": tool_call.id,  # Use the specific tool_call.id
                        "content": wiki_result
                    }
                    messages.append(tool_response)
            
            # Final API call
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                max_tokens=1000
            )
        
        # Get final answer
        final_answer = response.choices[0].message.content
        
        # Display with beautiful HTML
        display_answer_with_html(question, final_answer)
        
        return final_answer
        
    except Exception as e:
        error_msg = f"Error: {str(e)}"
        display_answer_with_html(question, error_msg)
        return error_msg

# Test the function with HTML display
result = answer_question("What is quantum computing?")

## Example: When No Tool is Needed

This example demonstrates when Claude answers directly without using external tools.


In [ ]:
# Question that doesn't require external data
question = "What is 2 + 2?"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": question}],
    max_tokens=500,
    tools=[article_search_tool]
)

print(f"Finish reason: {response.choices[0].finish_reason}")
print(f"\nGPT's response:")
if not response.choices[0].message.tool_calls:
    print("✅ Answered without tools:")
    print(response.choices[0].message.content)
else:
    print("⚠️  Used a tool (unexpected)")

## Multi-Turn Conversation with Tools

Demonstrates handling multiple tool calls in a single conversation.


In [ ]:
# Multi-turn conversation example
messages = [
    {"role": "user", "content": "Tell me about the movie Nezha 2"}
]

print("🔄 Starting multi-turn conversation...\n")

# First API call
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    max_tokens=1000,
    tools=[article_search_tool]
)

# Handle tool use
if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    tool_args = json.loads(tool_call.function.arguments)
    
    print(f"📚 GPT wants to search: '{tool_args['search_term']}'")
    
    # Add assistant's response (must include tool_calls)
    messages.append({
        "role": "assistant",
        "content": response.choices[0].message.content,
        "tool_calls": [{
            "id": tool_call.id,
            "type": "function",
            "function": {
                "name": tool_call.function.name,
                "arguments": tool_call.function.arguments
            }
        }]
    })
    
    # Execute tool
    wiki_result = get_article(tool_args["search_term"])
    
    # Add tool result
    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": wiki_result[:2000]  # Limit length
    })
    
    # Get final response
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=1000
    )
    
    print(f"\n💬 GPT's answer:")
    print(response.choices[0].message.content[:500] + "...")

## Handling Errors Gracefully

Examples of how the robust error handling works.


In [ ]:
# Example 1: Non-existent article
print("Test 1: Non-existent article")
result = get_article("xyzabc123notrealatall")
print(result[:200])

print("\n" + "="*50 + "\n")

# Example 2: Ambiguous search (will show disambiguation options)
print("Test 2: Ambiguous search term")
result = get_article("Mercury")
print(result[:300])

## Key Differences: OpenAI vs Anthropic Tool Use

### Schema Format
- **OpenAI**: Wraps in `type: "function"` with `parameters` field
```python
{"type": "function", "function": {"name": "...", "parameters": {...}}}
```
- **Anthropic**: Uses `input_schema` directly
```python
{"name": "...", "description": "...", "input_schema": {...}}
```

### Response Structure
- **OpenAI**: `response.choices[0].message.tool_calls` is array or None
- **Anthropic**: `response.content` is array of ContentBlocks

### Tool Result Format
- **OpenAI**: 
```python
{"role": "tool", "tool_call_id": "...", "content": "..."}
```
- **Anthropic**: 
```python
{"role": "user", "content": [{"type": "tool_result", ...}]}
```

### Finish/Stop Reasons
- **OpenAI**: `"tool_calls"`, `"stop"`, `"length"`, `"content_filter"`
- **Anthropic**: `"tool_use"`, `"end_turn"`, `"max_tokens"`


## 🎓 Summary & Next Steps

### What You Learned
- ✅ How to define tools for OpenAI GPT models
- ✅ Complete agentic loop with function calling API
- ✅ Error handling for edge cases
- ✅ Multi-turn conversations with tools
- ✅ Security best practices (json.loads vs eval)

### Best Practices
1. **Always validate inputs** before calling tools
2. **Use json.loads()** not eval() for parsing arguments
3. **Handle errors gracefully** with try/except blocks
4. **Use helper functions** (`extract_tool_calls`) for cleaner code
5. **Display results beautifully** with HTML formatting

### Resources
- [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)
- [OpenAI API Reference](https://platform.openai.com/docs/api-reference)
- [GitHub Repository](https://github.com/bleongcw/generativeai_course)

### Next Steps
1. Try building your own tools (API calls, database queries, etc.)
2. Experiment with multiple tools working together
3. Add streaming for real-time responses
4. Implement caching for frequently-used data
5. Build a complete application using these patterns
